<a href="https://colab.research.google.com/github/watanabe3tipapa/GoogleColaboratory/blob/main/EtudeJupyterDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 全体概要
1. ColabにてJupyterDashとBootStrapを使用するためのライブラリをインストールする
1. webアプリの基底クラスとしてJupyterDashを継承したクラスを作成する
  - webページのタイトル（タブに表示される名称）を設定する
  - BootStrapを使用可能にする
  - レイアウト（html+cssに相当）を設定する抽象メソッドを実行する
  - コールバック関数（動的な処理を追加、javascriptに相当）を設定する抽象メソッドを実行する
  - 上記抽象メソッド実行後、ポート番号を指定してwebアプリを起動する
1. 基底クラスを継承してwebアプリを定義したクラスを作成する
  - レイアウトを設定する抽象メソッドをオーバーライドする
  - コールバック関数を設定する抽象メソッドをオーバーライドする
1. タイトル名とポート番号を指定して実行する

# 実行する

1. ColabにてJupyterDashとBootStrapを使用するためのライブラリをインストールする

In [ ]:
# dashのインストール
!pip install dash==2.0.0

# jupyter-dashインストール
!pip install jupyter-dash==0.4.0

# dash_bootstrap_componentsインストール
!pip install dash_bootstrap_components==0.13.1

     |████████████████████████████████| 7.3 MB 5.4 MB/s 
     |████████████████████████████████| 357 kB 68.3 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=7defe7a2c70db5a1268c679829475f619c8784d3f0bb351296039244470d3eb8
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
     |████████████████████████████████| 197 kB 5.5 MB/s 


2. webアプリの基底クラスとしてJupyterDashを継承したクラスを作成する

In [ ]:
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc

class JupyterDashBootStrap(JupyterDash):
  def __init__(self, _title, _port):
    super().__init__(__name__, external_stylesheets=self.__external_stylesheets())
    if _title is not None:
      self.title = _title
    self.__setlayout()
    self.__run(_port)
  
  def __setlayout(self):
      self.layout = self.buildbaselayout()
      self.registcalback(self)

  def buildbaselayout(self):
      return
  
  def registcalback(self, app):
      return

  def __run(self, _port):
    if _port is None:
      self.run_server(mode="external")
    else:
      self.run_server(mode="external", port=_port )
  
  def __external_stylesheets(self):
    return [dbc.themes.BOOTSTRAP]

/usr/local/lib/python3.7/dist-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


3. 基底クラスを継承してwebアプリを定義したクラスを作成する

### Simple sidebar

In [ ]:
from dash import html
from dash.dependencies import Input, Output
from dash import dcc
from dash.dependencies import Input, Output
class MyDash(JupyterDashBootStrap):
  def buildbaselayout(self):
    # the style arguments for the sidebar. We use position:fixed and a fixed width
    SIDEBAR_STYLE = {
        "position": "fixed",
        "top": 0,
        "left": 0,
        "bottom": 0,
        "width": "16rem",
        "padding": "2rem 1rem",
        "background-color": "#f8f9fa",
    }

    # the styles for the main content position it to the right of the sidebar and
    # add some padding.
    CONTENT_STYLE = {
        "margin-left": "18rem",
        "margin-right": "2rem",
        "padding": "2rem 1rem",
    }

    sidebar = html.Div(
        [
            html.H2("Sidebar", className="display-4"),
            html.Hr(),
            html.P(
                "A simple sidebar layout with navigation links", className="lead"
            ),
            dbc.Nav(
                [
                    dbc.NavLink("Home", href="/", active="exact"),
                    dbc.NavLink("Page 1", href="/page-1", active="exact"),
                    dbc.NavLink("Page 2", href="/page-2", active="exact"),
                    dbc.NavLink("Page 3", href="/page-3", active="exact"),
                ],
                vertical=True,
                pills=True,
            ),
        ],
        style=SIDEBAR_STYLE,
    )

    content = html.Div(id="page-content", style=CONTENT_STYLE)
    return html.Div([dcc.Location(id="url"), sidebar, content])

  def registcalback(self, app):
    @app.callback(
        Output("page-content", "children"), 
        [Input("url", "pathname")]
        )
    def render_page_content(pathname):
      print()
      if pathname == "/":
          return html.P("This is the content of the Google Colaboratory")
      elif pathname == "/page-1":
          return html.P("This is the content of page 1. Hello World!")
      elif pathname == "/page-2":
          return html.P("Oh cool, this is page 2!")
      elif pathname == "/page-3":
          return html.P("Oh cool, this is page 3!")

      # If the user tries to reach a different page, return a 404 message
      return dbc.Jumbotron(
          [
              html.H1("404: Not found", className="text-danger"),
              html.Hr(),
              html.P(f"The pathname {pathname} was not recognised..."),
          ]
      )

### Graphs in Tabs

In [ ]:
import time
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import pandas as pd
from sklearn import datasets
from sklearn.cluster import KMeans

iris_raw = datasets.load_iris()
iris = pd.DataFrame(iris_raw["data"], columns=iris_raw["feature_names"])
class MyDash2(JupyterDashBootStrap):
  def buildbaselayout(self):
    layout = dbc.Container(
        [
            dcc.Store(id="store"),
            html.H1("Dynamically rendered tab content"),
            html.Hr(),
            dbc.Button(
                "Regenerate graphs",
                color="primary",
                block=True,
                id="button",
                className="mb-3",
            ),
            dbc.Tabs(
                [
                    dbc.Tab(label="Scatter", tab_id="scatter"),
                    dbc.Tab(label="Histograms", tab_id="histogram"),
                ],
                id="tabs",
                active_tab="scatter",
            ),
            html.Div(id="tab-content", className="p-4"),
        ]
    )

    return layout

  def registcalback(self, app):
    @app.callback(
        Output("tab-content", "children"),
        [Input("tabs", "active_tab"), Input("store", "data")],
    )
    def render_tab_content(active_tab, data):
        """
        This callback takes the 'active_tab' property as input, as well as the
        stored graphs, and renders the tab content depending on what the value of
        'active_tab' is.
        """
        if active_tab and data is not None:
            if active_tab == "scatter":
                return dcc.Graph(figure=data["scatter"])
            elif active_tab == "histogram":
                return dbc.Row(
                    [
                        dbc.Col(dcc.Graph(figure=data["hist_1"]), width=6),
                        dbc.Col(dcc.Graph(figure=data["hist_2"]), width=6),
                    ]
                )
        return "No tab selected"


    @app.callback(Output("store", "data"), [Input("button", "n_clicks")])
    def generate_graphs(n):
        """
        This callback generates three simple graphs from random data.
        """
        if not n:
            # generate empty graphs when app loads
            return {k: go.Figure(data=[]) for k in ["scatter", "hist_1", "hist_2"]}

        # simulate expensive graph generation process
        time.sleep(2)

        # generate 100 multivariate normal samples
        data = np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 1]], 100)

        scatter = go.Figure(
            data=[go.Scatter(x=data[:, 0], y=data[:, 1], mode="markers")]
        )
        hist_1 = go.Figure(data=[go.Histogram(x=data[:, 0])])
        hist_2 = go.Figure(data=[go.Histogram(x=data[:, 1])])

        # save figures in a dictionary for sending to the dcc.Store
        return {"scatter": scatter, "hist_1": hist_1, "hist_2": hist_2}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until


4. タイトル名とポート番号を指定して実行する

In [ ]:
MyDash("*** Simple sidebar ***", 8881)
MyDash2("*** Graphs in Tabs ***", 8882)


Dash app running on:


<IPython.core.display.Javascript object>

Dash app running on:


<IPython.core.display.Javascript object>